# 아파트 실거래가 예측 - https://bit.ly/3v5edhJ

### 부동산 빅데이터와 AI를 이용하여 실거래가를 예측 분석 | 회귀 | RMSE

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
print(train_df.shape)
train_df.head()

(1216553, 13)


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [3]:
train_df = train_df.loc[train_df["transaction_id"]%50==0, :]
train_df = train_df.reset_index(drop=True)
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,50,12536,서울특별시,인의동,48-2,효성쥬얼리시티,인의동 48-2 효성쥬얼리시티,106.81,2006,200802,11~20,13,64000
2,100,3926,서울특별시,수송동,85,로얄팰리스스위트,수송동 85 로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
3,150,7094,서울특별시,숭인동,569-3,숭인동 105동 (569-3),숭인동 569-3 숭인동 105동 (569-3),59.74,2000,200803,21~31,2,18000
4,200,10147,서울특별시,창신동,703,창신쌍용2,창신동 703 창신쌍용2,106.62,1993,200804,11~20,5,43500


In [4]:
print(test_df.shape)
test_df.head()

(5463, 12)


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,부산광역시,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24320 entries, 0 to 24319
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          24320 non-null  int64  
 1   apartment_id            24320 non-null  int64  
 2   city                    24320 non-null  object 
 3   dong                    24320 non-null  object 
 4   jibun                   24320 non-null  object 
 5   apt                     24320 non-null  object 
 6   addr_kr                 24320 non-null  object 
 7   exclusive_use_area      24320 non-null  float64
 8   year_of_completion      24320 non-null  int64  
 9   transaction_year_month  24320 non-null  int64  
 10  transaction_date        24320 non-null  object 
 11  floor                   24320 non-null  int64  
 12  transaction_real_price  24320 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 2.4+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          5463 non-null   int64  
 1   apartment_id            5463 non-null   int64  
 2   city                    5463 non-null   object 
 3   dong                    5463 non-null   object 
 4   jibun                   5463 non-null   object 
 5   apt                     5463 non-null   object 
 6   addr_kr                 5463 non-null   object 
 7   exclusive_use_area      5463 non-null   float64
 8   year_of_completion      5463 non-null   int64  
 9   transaction_year_month  5463 non-null   int64  
 10  transaction_date        5463 non-null   object 
 11  floor                   5463 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 512.3+ KB


In [7]:
train_df["city"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

In [8]:
test_df["city"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

In [9]:
# train_df.loc[train_df["city"] == "서울특별시", "city"] = 0
# train_df.loc[train_df["city"] == "부산광역시", "city"] = 1
# train_df["city"] = train_df["city"].astype('int')

test_df.loc[test_df["city"] == "서울특별시", "city"] = 0
test_df.loc[test_df["city"] == "부산광역시", "city"] = 1
test_df["city"] = test_df["city"].astype('int')

In [10]:
a = {"서울특별시":0, "부산광역시":1}
train_df["city"] = train_df["city"].map(a)
train_df["city"].head()

0    0
1    0
2    0
3    0
4    0
Name: city, dtype: int64

In [11]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,50,12536,0,인의동,48-2,효성쥬얼리시티,인의동 48-2 효성쥬얼리시티,106.81,2006,200802,11~20,13,64000
2,100,3926,0,수송동,85,로얄팰리스스위트,수송동 85 로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
3,150,7094,0,숭인동,569-3,숭인동 105동 (569-3),숭인동 569-3 숭인동 105동 (569-3),59.74,2000,200803,21~31,2,18000
4,200,10147,0,창신동,703,창신쌍용2,창신동 703 창신쌍용2,106.62,1993,200804,11~20,5,43500


In [12]:
test_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,0,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,1,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,1,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,0,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,1,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


In [13]:
train_df.drop('addr_kr', axis=1, inplace=True)
test_df.drop('addr_kr', axis=1, inplace=True)

In [14]:
len(train_df["dong"].unique()), len(test_df["dong"].unique())

(408, 390)

In [15]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

In [16]:
tmp = pd.concat([train_df["dong"], test_df["dong"]])
tmp.head()

0    신교동
1    인의동
2    수송동
3    숭인동
4    창신동
Name: dong, dtype: object

In [17]:
le.fit(tmp)

LabelEncoder()

In [18]:
train_df["dong"] = le.transform(train_df["dong"])
test_df["dong"] = le.transform(test_df["dong"])

In [19]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,237,6-13,신현(101동),84.82,2002,200801,21~31,2,37500
1,50,12536,0,330,48-2,효성쥬얼리시티,106.81,2006,200802,11~20,13,64000
2,100,3926,0,227,85,로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
3,150,7094,0,233,569-3,숭인동 105동 (569-3),59.74,2000,200803,21~31,2,18000
4,200,10147,0,368,703,창신쌍용2,106.62,1993,200804,11~20,5,43500


In [20]:
len(train_df["apartment_id"].unique()), len(train_df["apt"].unique())

(5753, 4770)

In [21]:
train_df[train_df["apartment_id"]==7622]

,transaction_id,apartment_id,city,dong,jibun,apt,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,237,6-13,신현(101동),84.82,2002,200801,21~31,2,37500


In [22]:
train_df[train_df["apartment_id"]==7623]

,transaction_id,apartment_id,city,dong,jibun,apt,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price


In [23]:
apt_name = train_df['apt']

for i in range(len(apt_name)):
    if i%500==0:
        print(apt_name[i])

신현(101동)
대림강변
와이엠프라젠스파
미주
한신1차
길음동동부센트레빌(1278-0)
창동주공18단지
한양1
하계1청구
성원(사슴4)
동부그린B
남가좌삼성
상록
목동신시가지5
화곡예다움
우장산힐스테이트
구일우성
시범
주공2
관악동부센트레빌
신반포4
서초포레스타1단지
한신(개포)
훼밀리
성원상떼빌
주공3
상상프리
에덴금호
서면한신
개금주공2
반도보라스카이뷰
삼성
벽산삼협한솔
롯데낙천대
금호
프린스빌라
건영
국제그린
크로바
구서동롯데캐슬골드2단지(1049)
조은빌라
삼익비치
신구덕우성
태영
래미안 옥수 리버젠
불암현대
남서울건영2
현대1차
센텀우신골든빌


In [24]:
train_df.drop("apt", axis=1, inplace=True)
test_df.drop("apt", axis=1, inplace=True)

train_df.drop("jibun", axis=1, inplace=True)
test_df.drop("jibun", axis=1, inplace=True)

In [25]:
train_df.head()

,transaction_id,apartment_id,city,dong,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,237,84.82,2002,200801,21~31,2,37500
1,50,12536,0,330,106.81,2006,200802,11~20,13,64000
2,100,3926,0,227,40.07,2005,200803,11~20,10,19000
3,150,7094,0,233,59.74,2000,200803,21~31,2,18000
4,200,10147,0,368,106.62,1993,200804,11~20,5,43500


In [26]:
train_df['transaction_date'].unique(), test_df['transaction_date'].unique()

(array(['21~31', '11~20', '1~10', '21~30', '21~28', '21~29'], dtype=object),
 array(['11~20', '21~31', '1~10', '21~30', '21~28'], dtype=object))

In [27]:
date_range = {'1~10': 0, '11~20': 1, '21~28': 3, '21~29': 3, '21~30': 3, '21~31': 3}

train_df.replace({"transaction_date": date_range}, inplace=True)
test_df.replace({"transaction_date": date_range}, inplace=True)

train_df.head()

,transaction_id,apartment_id,city,dong,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,237,84.82,2002,200801,3,2,37500
1,50,12536,0,330,106.81,2006,200802,1,13,64000
2,100,3926,0,227,40.07,2005,200803,1,10,19000
3,150,7094,0,233,59.74,2000,200803,3,2,18000
4,200,10147,0,368,106.62,1993,200804,1,5,43500


In [28]:
train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24320 entries, 0 to 24319
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          24320 non-null  int64  
 1   apartment_id            24320 non-null  int64  
 2   city                    24320 non-null  int64  
 3   dong                    24320 non-null  int64  
 4   exclusive_use_area      24320 non-null  float64
 5   year_of_completion      24320 non-null  int64  
 6   transaction_year_month  24320 non-null  int64  
 7   transaction_date        24320 non-null  int64  
 8   floor                   24320 non-null  int64  
 9   transaction_real_price  24320 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 1.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -

(None, None)

In [29]:
from sklearn.model_selection import train_test_split

x = train_df.drop("transaction_real_price", axis=1)
y = train_df["transaction_real_price"]

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=42)

print(x_train.shape, x_valid.shape)
print(y_train.shape, y_valid.shape)

(17024, 9) (7296, 9)
(17024,) (7296,)


In [30]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train, y_train)

y_hat = lr.predict(x_valid)

/Users/yehoon/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [31]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_valid, y_hat)**0.5
rmse

31455.914421978054

In [33]:
y_test = pd.Series(lr.predict(test_df))

In [34]:
submission = pd.concat([test_df['transaction_id'], y_test], axis=1)

cols = ['transaction_id', 'transaction_real_price']
submission.columns = cols
submission.to_csv('submission.csv', header = True, index = False)

In [35]:
test = pd.read_csv('submission.csv')
test

,transaction_id,transaction_real_price
0,1145756,17000
1,1198704,11000
2,1222384,11000
3,1179897,17000
4,1223091,11000
...,...,...
5458,1174640,17000
5459,1175575,11000
5460,1157024,17000
5461,1136863,17000
